
## Getting a Player's Last Game Stats

In this notebook, we will demonstrate how to retrieve and analyze the statistics of a player's last game using the `nba_api` library. Specifically, we will focus on Shai Gilgeous-Alexander from the Oklahoma City Thunder. The steps include:

1. **Importing Required Libraries**: We will use the `nba_api` library to fetch player statistics.
2. **Defining Functions**:
    - `get_team_on_court_stats(player_id, team_id)`: Retrieves the on-court statistics for a given player and team.
    - `get_player_vs_player(player_id, vs_player_id)`: Compares statistics between two players.
3. **Fetching Data**: Using the defined functions to fetch and print the relevant statistics for an inputted player name


In [3]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
import pandas as pd

def get_player_last_game_stats(player_name):
    # Search for players matching the name
    player_list = players.find_players_by_full_name(player_name)
    
    if not player_list:
        print(f"No player found with the name {player_name}.")
        return
    
    # If multiple players are found, list them
    if len(player_list) > 1:
        print("Multiple players found:")
        for idx, player in enumerate(player_list):
            print(f"{idx + 1}. {player['full_name']} (ID: {player['id']})")
        try:
            choice = int(input("Select the player by number: ")) - 1
            selected_player = player_list[choice]
        except (IndexError, ValueError):
            print("Invalid selection.")
            return
    else:
        selected_player = player_list[0]
    
    player_id = selected_player['id']
    player_full_name = selected_player['full_name']
    
    # Update the season to the current NBA season
    season = '2024-25'  # Adjust this to the desired season
    gamelog = playergamelog.PlayerGameLog(player_id=player_id, season=season)
    
    # Convert the game logs to a pandas DataFrame
    gamelog_df = gamelog.get_data_frames()[0]
    
    if gamelog_df.empty:
        print(f"No game logs available for {player_full_name} in the {season} season.")
        return
    
    # Get the most recent game (first row)
    last_game = gamelog_df.iloc[0]
    
    # Display the stats
    print(f"\nLast game stats for {player_full_name}:")
    print(f"Date: {last_game['GAME_DATE']}")
    print(f"Matchup: {last_game['MATCHUP']}")
    print(f"Minutes Played: {last_game['MIN']}")
    print(f"Points: {last_game['PTS']}")
    print(f"Rebounds: {last_game['REB']}")
    print(f"Assists: {last_game['AST']}")
    print(f"Field Goal Percentage: {last_game['FG_PCT']}")
    print(f"3-Pointers Made: {last_game['FG3M']}")
    print(f"Steals: {last_game['STL']}")
    print(f"Blocks: {last_game['BLK']}")
    print(f"Turnovers: {last_game['TOV']}")

# Example usage
if __name__ == "__main__":
    player_name = input("Enter the player's name: ")
    get_player_last_game_stats(player_name)



Last game stats for James Harden:
Date: NOV 08, 2024
Matchup: LAC @ SAC
Minutes Played: 35
Points: 22
Rebounds: 4
Assists: 8
Field Goal Percentage: 0.313
3-Pointers Made: 3
Steals: 2
Blocks: 0
Turnovers: 7


In [12]:
# Import necessary libraries
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import players
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Function to fetch game logs for a player
def fetch_player_game_logs(player_id):
    gamefinder = leaguegamefinder.LeagueGameFinder(player_id_nullable=player_id)
    games = gamefinder.get_data_frames()[0]
    #print(gamefinder.get_data_frames())
    return games

# Function to prepare dataset for the model
def prepare_dataset(player_ids):
    data = pd.DataFrame()
    for player_id in player_ids:
        games = fetch_player_game_logs(player_id)
        # Ensure 'PLAYER_ID' is included in the DataFrame
        games['PLAYER_ID'] = player_id
        # Sort games by date
        games = games.sort_values('GAME_DATE')
        # Keep only necessary columns
        games = games[['PLAYER_ID', 'GAME_DATE', 'PTS']]
        # Calculate rolling window of last 5 games
        games['PTS_last1'] = games['PTS'].shift(1)
        games['PTS_last2'] = games['PTS'].shift(2)
        games['PTS_last3'] = games['PTS'].shift(3)
        games['PTS_last4'] = games['PTS'].shift(4)
        games['PTS_last5'] = games['PTS'].shift(5)
        # Drop rows with NaN values (first 5 games)
        games = games.dropna()
        data = pd.concat([data, games], ignore_index=True)
    return data

# Get player IDs for the players you want to include
# For example, let's use LeBron James and Stephen Curry
player_names = ['LeBron James', 'Stephen Curry']
player_ids = []
for name in player_names:
    player = players.find_players_by_full_name(name)
    if player:
        player_ids.append(player[0]['id'])
    else:
        print(f"Player {name} not found.")

# Prepare the dataset
data = prepare_dataset(player_ids)

# Define features and target variable
features = ['PTS_last1', 'PTS_last2', 'PTS_last3', 'PTS_last4', 'PTS_last5']
target = 'PTS'

# Split data into inputs (X) and outputs (y)
X = data[features].values
y = data[target].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# Feature Scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = Sequential()
model.add(Dense(64, input_dim=5, activation='relu'))  # Input layer with 5 features
model.add(Dense(32, activation='relu'))              # Hidden layer
model.add(Dense(1))                                  # Output layer for regression

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=8, validation_data=(X_test_scaled, y_test))

# Evaluate the model
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test Mean Absolute Error: {mae}")

# Predict on test data
y_pred = model.predict(X_test_scaled)

# Compare predictions with actual values
comparison = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
print(comparison.head())


Epoch 1/100


/home/limpyshrimpy/anaconda3/envs/cs584/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 480.4492 - mae: 18.8479 - val_loss: 71.3226 - val_mae: 6.5959
Epoch 2/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 76.0524 - mae: 6.8386 - val_loss: 71.5315 - val_mae: 6.5695
Epoch 3/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 71.9241 - mae: 6.7328 - val_loss: 70.5248 - val_mae: 6.5283
Epoch 4/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 75.5616 - mae: 6.9123 - val_loss: 70.8659 - val_mae: 6.5892
Epoch 5/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 75.3422 - mae: 6.8481 - val_loss: 70.5019 - val_mae: 6.5665
Epoch 6/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 71.4628 - mae: 6.6087 - val_loss: 70.9085 - val_mae: 6.6038
Epoch 7/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 70.3522 - mae: 6.5959 - val_loss: 70.2669 - val_mae: 6.5227
Epoch 8/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 74.3537 - mae: 6.8308 - val_loss: 69.8881 - val_mae: 6.5311
Epoch 9/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 0

In [24]:
# Function to predict a player's next game points
def predict_player_next_game(player_name):
    player_id = players.find_players_by_full_name(player_name)[0]['id']
    player_games = fetch_player_game_logs(player_id)
    player_games = player_games.sort_values('GAME_DATE')
    player_games = player_games[['PTS']].tail(5).values.flatten()

    # Ensure we have exactly 5 games to predict
    if len(player_games) == 5:
        player_games_scaled = scaler.transform([player_games])
        player_pred = model.predict(player_games_scaled)
        print(f"Predicted points for {player_name}'s next game: {player_pred[0][0]}")
    else:
        print(f"Not enough data to predict {player_name}'s next game.")

# Example usage
predict_player_next_game('Lauri Markkanen')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Predicted points for Lauri Markkanen's next game: 19.037328720092773


In [26]:
# Import necessary libraries
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import playergamelog, leaguedashteamstats
from nba_api.stats.static import teams, players

# Functions to fetch data
def get_team_defensive_ratings(season='2022-23'):
    team_stats = leaguedashteamstats.LeagueDashTeamStats(
        season=season,
        measure_type_detailed_defense='Advanced',
        per_mode_detailed='PerGame'
    )
    df = team_stats.get_data_frames()[0]
    df = df[['TEAM_ID', 'DEF_RATING']]
    return df

def fetch_player_game_logs(player_id, season):
    gamelog = playergamelog.PlayerGameLog(player_id=player_id, season=season)
    games = gamelog.get_data_frames()[0]
    return games

def prepare_dataset(player_ids, seasons):
    data = pd.DataFrame()
    team_def_ratings = get_team_defensive_ratings()
    for player_id in player_ids:
        for season in seasons:
            games = fetch_player_game_logs(player_id, season)
            # Ensure GAME_DATE is datetime
            games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
            games = games.sort_values('GAME_DATE')
            # Add shifted points
            # ...
            # Merge team defensive ratings
            games = games.merge(team_def_ratings, left_on='OPPONENT_TEAM_ID', right_on='TEAM_ID', how='left')
            # Add to the main dataset
            data = pd.concat([data, games], ignore_index=True)
    return data

# Prepare the dataset
player_ids = ['Lebron James']  # Replace with actual player IDs
seasons = ['2024-25']
data = prepare_dataset(player_ids, seasons)

# Define features including defensive rating
features = [
    'PTS_last1', 'PTS_last2', 'PTS_last3', 'PTS_last4', 'PTS_last5',
    'DEF_RATING'  # This is the opponent's defensive rating
    # Add other features as needed
]

# Continue with data preprocessing and model training


JSONDecodeError: Expecting value: line 1 column 1 (char 0)